# generating videos of motion
Tim Tyree<br>
12.6.2020

In [1]:
%matplotlib inline
from lib.my_initialization import *
from lib.controller.integrate_forward_dormand_prince_asynchronous import *
from lib.controller.integrate_forward_implicit_asynchronous import *
%autocall 1

%load_ext autoreload
%autoreload 2


Automatic calling is: Smart


# import mesh

In [2]:
#define Lamé parameters
mu = 10.; lam = 1.; 
#define coefficient for Rayleigh damping
gamma = 0.1#1.#1.;
#define adaptive time stepping absolute tolerances
salience = 1#128#32
learning_rate = np.log(salience) #np.log(16) #np.log(2)
lasso_fraction = 0.5#not used rn...
v_scale = 2.
mass_density=1.
stepsize_init = 0.001#0.00001  #0.0001
atol_x = 1e-05#0.0001#0.001#1e-7; 
atol_v = 1e-05#0.0001#0.001#1e-7; 
btol_x = 1e-07#0.00001#0.001#1e-10; 
btol_v = 1e-07#0.00001#0.001#1e-10; 
input_file_name = f'../data/spherical_meshes/spherical_mesh_64.stl'#path to mesh
data_folder =  os.path.join(nb_dir,'../data/mov_csv')#where to save results


data_fn = f"explicit_asynch_fixed_lr_{os.path.basename(input_file_name).replace('.stl',f'_mu_{mu}_lambda_{lam}_gamma_{gamma}vscale_{v_scale}_stepsizeinit_{stepsize_init}')}_salience_{salience}_atolx_{atol_x}.csv"
# data_fn = f"dormand_prince_asynch_fixed_lr_{os.path.basename(input_file_name).replace('.stl',f'_mu_{mu}_lambda_{lam}_gamma_{gamma}vscale_{v_scale}_stepsizeinit_{stepsize_init}')}_salience_{salience}_atolx_{atol_x}.csv"
TEXTfoo = lambda N_vertices,N_elements: f"""
TEXT=$'an inward squish
        Asynchronous Variational Integrator 
        Explicit Newmark Method
        
        Undamped Neohookean Model
        mu {mu}  lambda {lam} gamma {gamma}
        
        Spherical Mesh
        Num. Vertices {N_vertices}
        Num. Elements {N_elements}
'"""

# data_fn = f"avi_esynchronous_fixed_lr_a_equal_b_{os.path.basename(input_file_name).replace('.stl',f'_mu_{mu}_lambda_{lam}_gamma_{gamma}_vscale_{v_scale}_stepsizeinit_{stepsize_init}')}_salience_{salience}_atolx_{atol_x}.csv"
# data_fn = f"avi_ns_fixed_lr_a_equal_b_{os.path.basename(input_file_name).replace('.stl',f'_mu_{mu}_lambda_{lam}_gamma_{gamma}vscale_{v_scale}_stepsizeinit_{stepsize_init}')}_salience_{salience}_atolx_{atol_x}.csv"
# data_fn = f"dormand_prince_synch_fixed_lr_{os.path.basename(input_file_name).replace('.stl',f'_mu_{mu}_lambda_{lam}_gamma_{gamma}vscale_{v_scale}_stepsizeinit_{stepsize_init}')}_salience_{salience}_atolx_{atol_x}.csv"
save_folder_vid = '../vid/tmp2'
folder_vid = '../vid'
# data_fn_counts = data_fn.replace("s_","s_counts_").replace('.csv','.npz')
data_fn_counts = data_fn.replace("s_","s_counts_").replace('.csv','.npy')
os.chdir(nb_dir)
# input_file_name = f'../data/spherical_meshes/spherical_mesh_64.stl'input_file_name = f'../data/spherical_meshes/spherical_mesh_64.stl'
# input_file_name = f'../data/spherical_meshes/spherical_mesh_1000.stl'
input_file_name = os.path.join(nb_dir,input_file_name)
tme = 0.
dict_values_system = initialize_system(input_file_name, time_initial=tme, mass_density=mass_density)
locals().update(dict_values_system)
N_elements = element_array_index.shape[0]
N_vertices = node_array_position.shape[0]
TEXT = TEXTfoo(N_vertices,N_elements)
print(list(dict_values_system.keys()))

#header to start of video
# TEXT = '''TEXT=$'Synchronous Variational Integrator\nDamped Neohookean Model\n(inward squish)''''
# TEXT = f"""
# TEXT=$'an inward squish
#         Synchronous Variational Integrator 
#         Explicit Newmark Method
        
#         Undamped Neohookean Model
#         mu {mu}  lambda {lam} gamma {gamma}
        
#         Spherical Mesh
#         Num. Vertices {N_vertices}
#         Num. Elements {N_elements}
# '"""

#write header to file
# print(TEXT)
# Dormand–Prince Method
# Explicit Newmark Method
# Implicit Midpoint Rule

face_normals all zero, ignoring!


['element_array_time', 'element_array_index', 'element_array_mass', 'element_array_volume', 'element_array_inverse_equilibrium_position', 'node_array_equilibrium_position', 'node_array_time', 'node_array_position', 'node_array_momentum', 'node_array_mass', 'node_array_volume']


# Explicit/Implicit Synchronous/Asynchronous Variational Integrator

- Mesh = solid sphere discretized by tetrahedra 
- Forces = neohookean hyperelastic material with rayleigh damping
- Initialization = inward initial velocity by an amount proportional to the x coordinate.

In [3]:
#initialize system
tauK = element_array_time
tau = node_array_time

#get method of computing elastic forces 
zero_mat = np.zeros((4,3))
# calc_P = get_calc_P(mu, lam)
# compute_nodal_damping_forces  = get_compute_nodal_damping_forces(mu,lam,gamma)
# comp_nodal_elastic_forces = get_comp_nodal_elastic_forces(mu, lam)

elements = element_array_index
vertices = node_array_position

#initialize stepsizes of simulation
element_array_stepsize = np.zeros_like(element_array_time) + stepsize_init
element_array_count_calls_one_step = np.zeros_like(element_array_time,dtype=int)
element_array_count_config_updates = np.zeros_like(element_array_time,dtype=int)
momentum = node_array_momentum.copy()
velocities = momentum.copy()
for j in range(N_vertices):
    velocities[j] /= node_array_mass[j]


# #perturb momentum in the x direction and let it run overnight with a small timestep  
velocities[:,0] = -v_scale * vertices[:,0].copy()

#initialize containers of measures
volume_lst = []
energy_lst = []
tme_lst    = []
stepsize_mean_lst   = []
stepsize_std_lst    = []
stepsize_median_lst = []
stepsize_count_lst = []
frac_lb = lambda i: np.exp(learning_rate*i)  ##np.exp((i+.5)*learning_rate/10)
learning_bins = np.array([stepsize_init*frac_lb(i) for i in np.arange(-15,15)])

# learning_bins = np.array([stepsize_init*np.exp((i+.5)*learning_rate) for i in np.arange(-3,3)])
print(f"learning_bins are {learning_bins}")

learning_bins are [0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001]


In [4]:
#choose your fighter
# integrate_system_explicit_synchronous = get_integrate_system_explicit_synchronous(mu,lam,gamma)
integrate_system_explicit_asynchronous = get_integrate_system_explicit_asynchronous(mu,lam,gamma)
# integrate_system_implicit_synchronous = get_integrate_system_implicit_synchronous(mu,lam,gamma,num_iter=30)
# integrate_system_implicit_asynchronous = get_integrate_system_implicit_asynchronous(mu,lam,gamma)
# integrate_system_dormand_prince_synchronous = get_integrate_system_dormand_prince_synchronous(mu,lam,gamma)
# integrate_system_dormand_prince_asynchronous = get_integrate_system_dormand_prince_asynchronous(mu,lam,gamma,atol_x, atol_v, btol_x, btol_v,learning_rate, lasso_fraction)
# mode = 'fixed_lr'#'neural_lr'
# integrate_system_dormand_prince_asynchronous = get_integrate_system_dormand_prince_asynchronous(mu,lam,gamma,atol_x, atol_v, btol_x, btol_v,learning_rate, lasso_fraction)


#(optional) #view the mesh
# plot_mesh(vertices, input_file_name=input_file_name, darkmode=True)

In [5]:
# def get_integrate_system(mode):
#     if mode==0:
#         return integrate_system_explicit_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities,node_array_mass, element_array_inverse_equilibrium_position)
#     #     integrate_system_explicit_asynchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities, node_array_mass, element_array_inverse_equilibrium_position, atol_x, atol_v, btol_x, btol_v, learning_rate)
#     #     integrate_system_implicit_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities,node_array_mass, element_array_inverse_equilibrium_position)
#     #     integrate_system_implicit_asynchronous(tf, element_array_time, element_array_stepsize, node_array_time,      element_array_index, vertices, velocities, node_array_mass, element_array_inverse_equilibrium_position, atol_x, atol_v, btol_x, btol_v, learning_rate)
# #     integrate_system_dormand_prince_synchronous(tf,element_array_time,element_array_stepsize,node_array_time,element_array_index,vertices,velocities, node_array_mass,element_array_inverse_equilibrium_position,element_array_mass)
# #     integrate_system_dormand_prince_asynchronous(tf,element_array_time,element_array_stepsize,node_array_time,element_array_index,vertices,velocities, node_array_mass,element_array_inverse_equilibrium_position,element_array_mass)


In [6]:
#prepare for video
###CAUTION WHEN RUNNING IN PARALLEL
frameno = 1
time_between_observations = 0.01
time_of_next_observation = tme + time_between_observations
os.chdir(nb_dir)
os.chdir(folder_vid)
with open("mov/text.txt", "w") as file:
    file.write(TEXT)
    file.close()

In [7]:
os.chdir(nb_dir)
os.chdir(folder_vid)
folnm = os.path.basename(save_folder_vid)
shutil.rmtree(folnm)
os.mkdir(folnm)

In [8]:
# !ls

In [ ]:
#ready... get set... GO!
os.chdir(nb_dir)
os.chdir(save_folder_vid)
time_end_recording = 3#.2
while time_of_next_observation <= time_end_recording:
    tf = time_of_next_observation
    #integrate forward to the next time of observation
    #     integrate_system_explicit_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities,node_array_mass, element_array_inverse_equilibrium_position)
    integrate_system_explicit_asynchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities, node_array_mass, element_array_inverse_equilibrium_position, atol_x, atol_v, btol_x, btol_v, learning_rate)
    #     integrate_system_implicit_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities,node_array_mass, element_array_inverse_equilibrium_position)
    #     integrate_system_implicit_asynchronous(tf, element_array_time, element_array_stepsize, node_array_time,      element_array_index, vertices, velocities, node_array_mass, element_array_inverse_equilibrium_position, atol_x, atol_v, btol_x, btol_v, learning_rate)
    #     integrate_system_dormand_prince_synchronous(tf,element_array_time,element_array_stepsize,node_array_time,element_array_index,vertices,velocities, node_array_mass,element_array_inverse_equilibrium_position,element_array_mass)
    #     integrate_system_dormand_prince_asynchronous(tf,element_array_time,element_array_stepsize,node_array_time,element_array_index,vertices,velocities, node_array_mass,element_array_inverse_equilibrium_position,element_array_mass)

    #update a copy of all positions to the observation time using the current velocity
    x = vertices.copy()
    for a in range(N_vertices):
        x[a] += velocities[a] * (tf - tau[a])
    
    #measure observables
    ##mesh measures
    net_volume = compute_net_volume(x, element_array_index)
    net_energy = compute_net_energy(N_elements, element_array_mass, velocities, x, #vertices,
                                  element_array_index, element_array_inverse_equilibrium_position, mu, lam)
    ##stepsize measures
    stepsize_mean = np.mean(element_array_stepsize)
    stepsize_std = np.std(element_array_stepsize)
    stepsize_median = np.median(element_array_stepsize)

    #record observables
    volume_lst.append(net_volume)
    energy_lst.append(net_energy)
    tme = tf
    tme_lst.append(tme)
    stepsize_mean_lst.append(stepsize_mean)
    stepsize_std_lst.append(stepsize_std)
    stepsize_median_lst.append(stepsize_median)
    stepsize_count_lst.append(count_array(array = element_array_stepsize,bins=learning_bins))
    #record image of system
    try:
        img = get_img_of_system(vertices, input_file_name, darkmode = False, text=f'time={tf:.2f}')
    except:
        pass
    save_fn_img = f'img{frameno:09}.png'
    frameno += 1
    Img = Image.fromarray(img)
    Img.save(save_fn_img)
    del Img
    #increment time_of_next_observation
    time_of_next_observation += time_between_observations
    #TODO(later): do something that makes a boring video less boring
    #TODO(later): def element_array_count_calls_one_step to the call of integrate_system_.... i.e. Keep track of how many times each element is called to time step
    #TODO(later): keep track of how many times each element config is updated with element_array_count_config_updates
print(stepsize_mean, stepsize_std, stepsize_median)
#TODO: include a progress bar...? No. put in a function!
beep(3)

/Users/timothytyree/Documents/GitHub/avi/nb/lib/controller/integrate_forward_explicit_asynchronous.py:45: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  next_stepsize, retval, madval = step_forward_and_learn_simple(K_index, t_given, node_array_time, element_array_time, vertices, velocities, element_array_index,
/Users/timothytyree/Documents/GitHub/avi/nb/lib/measure/mesh_measures.py:35: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  element_array_energy = comp_element_array_energy(N_elements, element_array_mass, velocities, vertices,


In [ ]:
print(time_of_next_observation)
from scipy.stats.mstats import gmean
# print(np.hmean(element_array_stepsize), gmean(element_array_stepsize))
print(gmean(element_array_stepsize))
print(np.mean(element_array_stepsize))
print(stepsize_init)
tme

In [ ]:
fontsize=20
plt.figure(figsize=(9,6))
plt.hist(element_array_stepsize, bins=100)
# Add some text for labels, title and custom x-axis tick labels, etc.
plt.ylabel('freq. of time step size',fontsize=fontsize)
plt.xlabel('final time step size',fontsize=fontsize)

# ax.set_title('Scores by group and gender')
# plt.xticks(x)#,fontsize=fontsize)
# plt.xticklabels(labels,fontsize=fontsize)
# plt.legend(fontsize=fontsize-4)
# plt.ylim([0,150])
# ax.set_xlabel('time',**axkwargs)
# ax.set_ylabel('variance in step size',**axkwargs)
plt.tick_params(labelsize=fontsize, labelrotation=20)

# ax.set_xlabel('time',**axkwargs)
# ax.set_ylabel('volume / initial volume',**axkwargs)
# plt.tick_params(labelsize=fontsize)
plt.tight_layout()
plt.show()

In [ ]:
# img = get_img_of_system(vertices, input_file_name, darkmode = True, text=f'time={tf:.2f}')
# img
stepsize_init
# plt.tick_params?

__problem/bug confirmed__ integrate_system_dormand_prince_asynchronous is only decreasing the stepsize values.

 __Verified that__ all variables are changed by the integrator

__time is now being updated at the nodes__

# save csv of results

In [ ]:
df = pd.DataFrame({
    't':tme_lst,
    'volume':volume_lst,
    'energy':energy_lst,
    'stepsize_mean':stepsize_mean_lst,
    'stepsize_std':stepsize_std_lst,
    'stepsize_median':stepsize_median_lst,
})

In [ ]:
df.head()

In [ ]:
# x_values = df['t'].values
# # y_values = df['volume'].values
# y_values = df['energy'].values
# plt.plot(x_values, y_values)

In [ ]:
x_values = df['t'].values
# y_values = df['volume'].values
# y_values = df['energy'].values
plt.plot(x_values, df['volume'].values/(4*np.pi*1.**3/3), label='volume/initial volume')
plt.plot(x_values, df['energy'].values, label='energy')
plt.legend()
plt.xlabel('time')
plt.ylabel('measurement value')
plt.show()

In [ ]:
# data_fn = f"avi_ns_a_equal_b_{os.path.basename(input_file_name).replace('.stl',f'_mu_{mu}_lambda_{lam}_gamma_{gamma}vscale_{v_scale}_stepsizeinit_{stepsize_init}')}.csv"

os.chdir(data_folder)
df.to_csv(data_fn, index=False)

In [ ]:
print(f'results saved as {data_fn}')

In [ ]:
np.array(stepsize_count_lst)
# np.savez(data_fn_counts)
np.save(data_fn_counts, np.array(stepsize_count_lst))

In [ ]:
# set(stepsize_count_lst)
np.array(stepsize_count_lst[0])
np.array(stepsize_count_lst[-1])

# compile and save the movie using ffmpeg

In [ ]:
os.chdir(nb_dir)
os.chdir(os.path.dirname(save_folder_vid))
vid_in_fn = 'out.mov'
vid_out_fn = data_fn.replace('.csv','.mov')

In [ ]:
!./concat_folder_png.sh

In [ ]:
os.chdir(nb_dir)
os.chdir(os.path.dirname(save_folder_vid))
assert(os.path.exists(vid_in_fn))
destination = shutil.copyfile(vid_in_fn, vid_out_fn) 
beep(2)

In [ ]:
os.path.exists(vid_out_fn)

In [ ]:
stepsize_mean